## Imports

In [ ]:
import evalseg

import numpy as np
import pandas as pd
from ipywidgets import interact,interact_manual, IntSlider
import os

## Variables

In [ ]:
root_data='../datasets'

## Metric Function

## Run Metric

In [ ]:
evalseg.reload()
@interact
def select_dataset(dataset_name=sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')])):
    global dataset

    dataset=evalseg.io.NibDataset(f'{root_data}/{dataset_name}')
    
    @interact_manual
    def select_case(case=sorted(dataset.get_available_ids())):
        gto,gto_voxelsize=dataset.get_groundtruth(case)
        cto,cto_voxelsize=dataset.get_CT(case)
        @interact
        def ax_select(ax=['z', 'x', 'y', 'all']):
            gtmax = (gto.sum(axis=2).sum(axis=1).argmax(), gto.sum(axis=2).sum(axis=0).argmax(), gto.sum(axis=1).sum(axis=0).argmax())
            axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]
            @interact
            def onlyMaxFrame(frame=IntSlider(-1,min=-1,max=-1) if ax=='all' else IntSlider(gtmax[axi], min=0, max=gto.shape[axi]-1)):
                global data, gt, voxelsize
                
                gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
                ct, _ = evalseg.geometry.slice(cto, None, axi, [frame])            
                mme = evalseg.metrics.MultiClassMetric(
                    evalseg.metrics.MME,
                    dataset.num_labels,
                    debug={'U':1,'B':1,'D':1,'UI':1}
                    )
                mme.set_reference(gt, voxelsize)

                @interact
                def pred_selec(prediction=dataset.get_prediction_methods()):
                    pred,_= dataset.get_prediction(prediction,case)
                    pred,_=evalseg.geometry.slice(pred, None, axi, [frame])
                    if ax!='all':
                        imglbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
                        comps=mme.metrics[1].helper['components']
                        gt2=gt.copy()
                        for c in comps:
                            gt2 = gt2& ~ comps[c]['gt_skeleton']
                        evalseg.ui.multi_plot_2d(ct, gt2, {prediction:pred}, spacing=voxelsize, args={"imglabel": imglbl, 'z_titles': [frame],'crop2roi':0})
                    else:
                        pass #TODO

                    res=mme.evaluate(pred,parallel=0)
                    evalseg.ui.plot_metric(res, name=dataset_name)


In [ ]:
evalseg.reload()
mme = evalseg.metrics.MultiClassMetric(
                    evalseg.metrics.MME,
                    dataset.num_labels,
                    debug={'U':1,'B':1,'D':1,'UI':1}
                    )
mme.set_reference(gt, voxelsize)

# mme.metrics[1].helper['components'][1]['skel_dst']


In [ ]:
mme.metrics[1].helper['components'][1]['skel_dst'].max()

In [ ]:
evalseg.reload()
fig=evalseg.ui.multi_plot_3d(mme.metrics[1].helper['components'][1],interactive=1)

In [ ]:

# fig.data[0].zmax=1
# fig.data[1].zmax=1
# fig.update_traces(zmin=0, selector=dict(type='heatmap'))
# fig.update_traces(zmax=1, selector=dict(type='heatmap'))
fig.update_traces(coloraxis=None, selector=dict(type='heatmap'))

fig.show()